In [3]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [4]:
from data.election_results_ingest import IngestElectionResults
from elections.elections import Elections
from pathlib import Path
import geopandas as gpd
import folium

In [91]:
ier = IngestElectionResults('~/Documents/data')

In [93]:
ier.ingest('../resources/fulton_20220524.xml')

In [94]:
ier.ingest('../resources/gwinnett_20220524.xml')

In [95]:
ier.ingest('../resources/forsyth_20220524.xml')

In [161]:
e = Elections('~/Documents/data')

In [99]:
df1 = e.get_results('2022-05-24', contest_pattern=r'US HOUSE\b.*\b7\b', party='REP',
                    other_pattern='MCCOY|WEST|NYGHTSTORM', level='state')

In [100]:
df1.to_csv('../workproducts/us_house_district_7_summary.csv')

In [101]:
df2 = e.get_results('2022-05-24', contest_pattern=r'US HOUSE\b.*\b7\b', party='REP',
                    other_pattern='MCCOY|WEST|NYGHTSTORM', level='county')

In [102]:
df2.to_csv('../workproducts/us_house_district_7_county_summary.csv')

In [103]:
df3 = e.get_over_under('2022-05-24', contest_pattern=r'US HOUSE\b.*\b7\b', level='county')

In [104]:
df3.to_csv('../workproducts/us_house_district_7_over_under_county_summary.csv')

In [105]:
df4 = e.get_over_under('2022-05-24', contest_pattern=r'US HOUSE\b.*\b7\b').sort_values(by='undervotes',
                                                                                       ascending=False).reset_index()

In [106]:
df4.to_csv('../workproducts/us_house_district_7_over_under.csv')

In [205]:
df5 = e.get_results('2022-05-24', contest_pattern=r'US HOUSE\b.*\b7\b', party='REP',
                    other_pattern='MCCOY|WEST|NYGHTSTORM').sort_values(by='votes', ascending=False).reset_index()


In [274]:
stats = df5[['choice', 'precinct_name', 'county', 'votes']].pivot(index=['precinct_name', 'county'], columns='choice', values='votes').reset_index()
stats = stats.assign(share=stats.iloc[:, 2]/(stats.iloc[:, 2]+stats.iloc[:, 3]+stats.iloc[:, 2])*100)

In [108]:
df5.to_csv('../workproducts/us_house_district_7.csv')

In [281]:
p = Path('~/Documents/data/ga/politics/maps/cng-prop1-2021-shape/districts.shp').expanduser()
us_house_map = gpd.read_file(p).to_crs(epsg=3035)
us_house_district_7_map = us_house_map[us_house_map.DISTRICT == '007'].reset_index()
center = us_house_district_7_map.centroid.to_crs(epsg=4326)
p = Path('~/Documents/data/ga/politics/maps/vtd/VTDlatest-Shapefile/VTD2020-Shapefile.shp').expanduser()
precincts = gpd.read_file(p)[['ID', 'PRECINCT_N', 'CTYNAME', 'geometry']].to_crs(epsg=3035)
precincts = precincts.sjoin(us_house_district_7_map[['geometry']]).drop(columns=['index_right'])
precincts = precincts.overlay(us_house_district_7_map[['geometry']], how='intersection', keep_geom_type=False)
precincts = precincts[(precincts.CTYNAME == 'FULTON') | (precincts.CTYNAME == 'GWINNETT')].reset_index()
precincts = precincts[['ID', 'PRECINCT_N', 'CTYNAME', 'geometry']]
precincts = precincts[precincts.PRECINCT_N != 'SUWANEE G'].reset_index(drop=True).to_crs(epsg=4326)
precincts.columns = ['id', 'precinct_name', 'county', 'geometry']
precinct_stats = precincts.merge(stats, on=['precinct_name', 'county'], how='inner')
precinct_stats = precinct_stats.assign(key=precinct_stats.precinct_name+precinct_stats.county)

In [282]:
precinct_stats = precinct_stats.assign(share1=precinct_stats.share.apply(lambda x: f'{x:.1f}%'))
precinct_stats.columns = ['id', 'Precinct Name', 'county', 'geometry', 'Gonsalves', 'Corbin', 'Other', 'share', "key", 'Gonsalves Share']


In [283]:
m = folium.Map(location=[center[0].y, center[0].x])

cp = folium.Choropleth(
    geo_data=precinct_stats,
    name="choropleth",
    data=precinct_stats,
    columns=["key", "share"],
    key_on="feature.properties.key",
    fill_color="YlGn",
    fill_opacity=0.5,
    line_opacity=1,
    line_weight=1.5,
    line_color='darkgray',
    legend_name="Gonsalves Share of Vote (%)",
).add_to(m)

folium.GeoJsonTooltip(['Precinct Name', 'Gonsalves', "Corbin", 'Other', 'Gonsalves Share']).add_to(cp.geojson)
m

/usr/local/lib/python3.9/site-packages/folium/features.py:906: UserWarning: GeoJsonTooltip is not configured to render for GeoJson GeometryCollection geometries. Please consider reworking these features: [{'Corbin': 13, 'Gonsalves': 7, 'Gonsalves Share': '25.9%', 'Other': 7, 'Precinct Name': 'AP14', 'county': 'FULTON', 'id': 116743, 'key': 'AP14FULTON', 'share': 25.925925925925924}] to MultiPolygon for full functionality.
https://tools.ietf.org/html/rfc7946#page-9
  warnings.warn(


In [284]:
m.save("gonsalves.html")

/usr/local/lib/python3.9/site-packages/folium/features.py:906: UserWarning: GeoJsonTooltip is not configured to render for GeoJson GeometryCollection geometries. Please consider reworking these features: [{'Corbin': 13, 'Gonsalves': 7, 'Gonsalves Share': '25.9%', 'Other': 7, 'Precinct Name': 'AP14', 'county': 'FULTON', 'id': 116743, 'key': 'AP14FULTON', 'share': 25.925925925925924}] to MultiPolygon for full functionality.
https://tools.ietf.org/html/rfc7946#page-9
  warnings.warn(


In [221]:
cp.geojson.data['features']

[{'id': '0',
  'type': 'Feature',
  'properties': {'MARK GONSALVES': 31,
   'MICHAEL CORBIN': 90,
   'OTHER': 74,
   'county': 'GWINNETT',
   'id': 3585,
   'precinct_name': 'ROCKBRIDGE A',
   'share': 0.20394736842105263},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-84.027987, 33.80424399001661],
     [-84.027989, 33.804198990016594],
     [-84.02797299999997, 33.80413299001657],
     [-84.02791000000002, 33.80406699001651],
     [-84.02778699999999, 33.803968990016514],
     [-84.027765, 33.80393399001643],
     [-84.027744, 33.80383799001644],
     [-84.02770099999998, 33.8033619900162],
     [-84.02772799999998, 33.803267990016124],
     [-84.027921, 33.80285799001592],
     [-84.02794299999998, 33.80275399001586],
     [-84.02800199999999, 33.802594990015756],
     [-84.02812399999999, 33.80237899001565],
     [-84.02822299999998, 33.80226099001563],
     [-84.02826900000001, 33.80216199001557],
     [-84.028293, 33.80195299001543],
     [-84.028351, 33.80185899001538]

In [122]:
sum(precincts.to_crs(epsg=3035).area < 100)

24

In [193]:
precincts

,,id,geometry
precinct_name,county,,
ROCKBRIDGE A,GWINNETT,3585,"POLYGON ((-84.02799 33.80424, -84.02799 33.804..."
ROCKBRIDGE C,GWINNETT,3586,"POLYGON ((-84.03612 33.80447, -84.03654 33.804..."
DULUTH H,GWINNETT,4097,"POLYGON ((-84.18011 34.00722, -84.18006 34.007..."
JC02,FULTON,4098,"POLYGON ((-84.14793 34.05729, -84.14782 34.057..."
BERKSHIRE G,GWINNETT,97539,"POLYGON ((-84.14731 33.84191, -84.14801 33.842..."
...,...,...,...
CATES G,GWINNETT,157187,"POLYGON ((-84.03274 33.86906, -84.03315 33.869..."
CATES B,GWINNETT,157188,"POLYGON ((-84.07572 33.89600, -84.07557 33.896..."
CATES K,GWINNETT,157189,"POLYGON ((-84.06829 33.90214, -84.06832 33.902..."


In [195]:
df5_precincts

,
precinct_name,county
HARBINS A,GWINNETT
JC12,FULTON
HARBINS B,GWINNETT
BAYCREEK H,GWINNETT
PINCKNEYVILLE M,GWINNETT
...,...
PINCKNEYVILLE J,GWINNETT
AP14,FULTON
JC04B,FULTON


In [204]:
df5[['choice', 'precinct_name', 'county', 'votes']].pivot(index=['precinct_name', 'county'], columns='choice', values='votes').reset_index()

choice,precinct_name,county,MARK GONSALVES,MICHAEL CORBIN,OTHER
0,AP14,FULTON,7,13,7
1,AP14A,FULTON,76,169,160
2,BAYCREEK A,GWINNETT,94,202,138
3,BAYCREEK B,GWINNETT,72,232,109
4,BAYCREEK C,GWINNETT,122,291,195
...,...,...,...,...,...
138,RW21A,FULTON,165,203,191
139,RW22B,FULTON,0,0,0
140,RW22C,FULTON,0,1,0
141,SS01A,FULTON,35,95,77


In [186]:
ep = df5.precinct_name

In [176]:
ep

array(['HARBINS A', 'JC12', 'HARBINS B', 'BAYCREEK H', 'PINCKNEYVILLE M',
       'PINCKNEYVILLE H', 'PINCKNEYVILLE L', 'JC11', 'BAYCREEK C',
       'BAYCREEK D', 'CATES M', 'GARNERS A', 'HARBINS C',
       'PINCKNEYVILLE Z', 'DULUTH I', 'BAYCREEK G', 'BERKSHIRE N',
       'DACULA', 'CATES G', 'DULUTH A', 'CATES N', 'LAWRENCEVILLE L',
       'BAYCREEK B', 'PINCKNEYVILLE A1', 'PINCKNEYVILLE W', 'BAYCREEK I',
       'DULUTH B', 'CATES D', 'RW21A', 'BAYCREEK A', 'JC07',
       'BERKSHIRE O', 'JC01', 'JC18', 'BERKSHIRE J', 'CATES A', 'JC04A',
       'CATES L', 'CATES I', 'LAWRENCEVILLE A', 'DULUTH G', 'CATES F',
       'BAYCREEK K', 'PINCKNEYVILLE X', 'CATES E', 'JC13A', 'BAYCREEK F',
       'CATES J', 'AP14A', 'CATES K', 'JC06', 'DULUTH D', 'JC16',
       'BAYCREEK J', 'BERKSHIRE D', 'LAWRENCEVILLE E', 'LAWRENCEVILLE B',
       'MARTINS H', 'JC19A', 'JC14', 'BERKSHIRE E', 'PINCKNEYVILLE C',
       'JC08', 'BERKSHIRE G', 'BAYCREEK E', 'CATES H', 'JC02', 'DULUTH E',
       'ROCKBRIDGE F', 'D

In [185]:
vp = precincts.precinct_name

In [183]:
vp

array(['ROCKBRIDGE A', 'ROCKBRIDGE C', 'DULUTH H', 'JC02', 'BERKSHIRE G',
       'GARNERS F', 'BERKSHIRE N', 'BAYCREEK E', 'BAYCREEK G',
       'BAYCREEK H', 'BAYCREEK D', 'BAYCREEK I', 'BAYCREEK C',
       'LAWRENCEVILLE D', 'DACULA', 'HARBINS B', 'HARBINS A', 'HARBINS C',
       'JC18', 'JC16', 'JC14', 'AP14', 'JC15', 'JC10', 'DULUTH E',
       'DULUTH C', 'JC11', 'JC09', 'JC07', 'JC08', 'JC05', 'JC04B',
       'JC03A', 'JC04A', 'JC03B', 'JC06', 'DULUTH A', 'JC01', 'MARTINS A',
       'CATES C', 'LAWRENCEVILLE F', 'LAWRENCEVILLE M', 'JC04C',
       'DULUTH B', 'DULUTH G', 'MARTINS E', 'DULUTH F', 'DULUTH I',
       'JC01A', 'PINCKNEYVILLE S', 'BERKSHIRE H', 'PINCKNEYVILLE Y',
       'PINCKNEYVILLE D', 'PINCKNEYVILLE K', 'PINCKNEYVILLE I', 'CATES A',
       'CATES L', 'BAYCREEK J', 'BAYCREEK B', 'BAYCREEK F', 'CATES N',
       'BAYCREEK A', 'PINCKNEYVILLE X', 'PINCKNEYVILLE H', 'JC12',
       'PINCKNEYVILLE T', 'DULUTH D', 'CATES I', 'CATES M',
       'LAWRENCEVILLE J', 'MARTINS B', '

In [200]:
idx = df5_precincts.index.difference(precincts.index)

In [208]:
stats

choice,precinct_name,county,MARK GONSALVES,MICHAEL CORBIN,OTHER,share
0,AP14,FULTON,7,13,7,0.259259
1,AP14A,FULTON,76,169,160,0.236760
2,BAYCREEK A,GWINNETT,94,202,138,0.241026
3,BAYCREEK B,GWINNETT,72,232,109,0.191489
4,BAYCREEK C,GWINNETT,122,291,195,0.228037
...,...,...,...,...,...,...
138,RW21A,FULTON,165,203,191,0.309568
139,RW22B,FULTON,0,0,0,NaN
140,RW22C,FULTON,0,1,0,0.000000
141,SS01A,FULTON,35,95,77,0.212121


In [203]:
idx

MultiIndex([(          'AP14A',   'FULTON'),
            (          'JC19A',   'FULTON'),
            ('PINCKNEYVILLE A', 'GWINNETT'),
            (          'RW21A',   'FULTON'),
            (          'RW22B',   'FULTON'),
            (          'RW22C',   'FULTON'),
            (          'SS01A',   'FULTON'),
            (      'SUWANEE G', 'GWINNETT')],
           names=['precinct_name', 'county'])

In [ ]:
address_cng =